In [1]:
import pandas as pd
from datetime import date, timedelta
import io
import requests

In [2]:
def request_file(date):
        URL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"
        global last_updated

        try:
            res = requests.get(URL + date.strftime("%m-%d-%Y") + ".csv")
            print("Request status code:", res.status_code)
            last_updated = date
            error = {"status": False, "message": ""}
            return res, error
        except:
            error = {"status": True, "message": "Error retrieving file from GitHub"}
            print(error["message"])
            return False, error

In [3]:
req, error = request_file(date.today())

Request status code: 404


In [4]:
if req.status_code == 404:
    req, error = request_file(date.today() - timedelta(days = 1))

Request status code: 200


In [5]:
df = pd.read_csv(io.StringIO(req.content.decode('utf-8')))
df.rename(columns={'Admin2': 'County', 'Province_State': 'State', 'Country_Region': 'Country',
                   'Last_Update': 'Update', 'Long_': 'Long', 'Combined_Key': 'Key'}, inplace=True)
df.set_index('Key', inplace=True)
for column in ["County", "State"]:
    df[column] = df[column].str.replace(' ','-')
    df[column] = df[column].str.lower()
df = df[df["Country"] == "US"]
df.to_csv('data.csv')

In [6]:
df.head()

,FIPS,County,State,Country,Update,Lat,Long,Confirmed,Deaths,Recovered,Active
Key,,,,,,,,,,,
"Abbeville, South Carolina, US",45001.0,abbeville,south-carolina,US,4/6/20 23:22,34.223334,-82.461707,6,0,0,0
"Acadia, Louisiana, US",22001.0,acadia,louisiana,US,4/6/20 23:22,30.295065,-92.414197,79,2,0,0
"Accomack, Virginia, US",51001.0,accomack,virginia,US,4/6/20 23:22,37.767072,-75.632346,11,0,0,0
"Ada, Idaho, US",16001.0,ada,idaho,US,4/6/20 23:22,43.452658,-116.241552,402,3,0,0
"Adair, Iowa, US",19001.0,adair,iowa,US,4/6/20 23:22,41.330756,-94.471059,1,0,0,0


In [7]:
# df[df.duplicated(['County'])]
# County is not unique, need combination of state and county

In [8]:
df['Confirmed'].sum()

366667

In [9]:
df[(df["State"] == "new-york")][["Confirmed", "Deaths", "Recovered", "Active"]].head()

,Confirmed,Deaths,Recovered,Active
Key,,,,
"Albany, New York, US",319,8,0,0
"Allegany, New York, US",17,1,0,0
"Broome, New York, US",76,5,0,0
"Cattaraugus, New York, US",12,0,0,0
"Cayuga, New York, US",11,0,0,0


In [10]:
query_state = "california"
query_county = "alameda"

In [11]:
query = df[(df["State"] == query_state) & (df["County"] == query_county)][["Confirmed", "Deaths", "Recovered", "Active"]]

In [12]:
query = query.to_dict(orient = "records")

In [13]:
query[0].update({"last_updated": "XXX"})

In [14]:
query

[{'Confirmed': 588,
  'Deaths': 13,
  'Recovered': 0,
  'Active': 0,
  'last_updated': 'XXX'}]

In [15]:
# Get list of state and counties
counties = df[["State", "County"]].sort_values(by=["State", "County"], ascending=True)
counties.to_csv('counties.csv', index = False)